In [1]:
%config IPCompleter.greedy=True
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import os

import spacy, pickle

import torch

from torchtext import data
from torchtext import datasets

import random
import inspect

In [3]:
path = "./"
save_data_path = path + 'save_data/'
large_save_data_path = '/notebooks/large-storage/'
print(os.listdir(path))

['test.tsv', 'prepare-word-embedding-nlp.ipynb', 'test-batching-padding-Copy1.ipynb', 'tokenization.ipynb', 'test-batching-padding.ipynb', 'train.tsv', 'sampleSubmission.csv', 'save_data', '.ipynb_checkpoints']


In [4]:
loaded_data = pickle.load(open(save_data_path + 'pre-processed-data.pkl', 'rb'))
loaded_kaggle_test = pickle.load(open(save_data_path + 'pre-processed-kaggle-test.pkl', 'rb'))
loaded_vocab = pickle.load(open(save_data_path + 'genereated-vocab.pkl', 'rb'))

In [5]:
loaded_data.head()

,PhraseId,SentenceId,Phrase,Sentiment,Phrase_length,Tokenized_phrase,Indexed_phrase
0,1,1,A series of escapades demonstrating the adage ...,1,188,"[xxbos, a, series, of, escapades, demonstratin...","[2, 10, 341, 11, 14246, 6044, 8, 6604, 19, 64,..."
1,2,1,A series of escapades demonstrating the adage ...,2,77,"[xxbos, a, series, of, escapades, demonstratin...","[2, 10, 341, 11, 14246, 6044, 8, 6604, 19, 64,..."
2,3,1,A series,2,8,"[xxbos, a, series, xxeos]","[2, 10, 341, 3]"
3,4,1,A,2,1,"[xxbos, a, xxeos]","[2, 10, 3]"
4,5,1,series,2,6,"[xxbos, series, xxeos]","[2, 341, 3]"


In [6]:
from torchtext.data.utils import get_tokenizer, dtype_to_attr, is_tokenizer_serializable

class CustomField(data.Field):
    def __init__(self, sequential=True, use_vocab=True, init_token=None,
                 eos_token=None, fix_length=None, dtype=torch.long,
                 preprocessing=None, postprocessing=None, lower=False,
                 tokenize=None, tokenizer_language='en', include_lengths=False,
                 batch_first=False, pad_token="<pad>", unk_token="<unk>",
                 pad_first=False, truncate_first=False, stop_words=None,
                 is_target=False):
        self.sequential = sequential
        self.use_vocab = use_vocab
        self.init_token = init_token
        self.eos_token = eos_token
        self.unk_token = unk_token
        self.fix_length = fix_length
        self.dtype = dtype
        self.preprocessing = preprocessing
        self.postprocessing = postprocessing
        self.lower = lower
        # store params to construct tokenizer for serialization
        # in case the tokenizer isn't picklable (e.g. spacy)
        self.tokenizer_args = (tokenize, tokenizer_language)
        self.tokenize = get_tokenizer(tokenize, tokenizer_language)
        self.include_lengths = include_lengths
        self.batch_first = batch_first
        self.pad_token = pad_token
        self.pad_first = pad_first
        self.truncate_first = truncate_first
        try:
            self.stop_words = set(stop_words) if stop_words is not None else None
        except TypeError:
            raise ValueError("Stop words must be convertible to a set")
        self.is_target = is_target
        
    def pad(self, minibatch):
        """Pad a batch of examples using this field.

        Pads to self.fix_length if provided, otherwise pads to the length of
        the longest example in the batch. Prepends self.init_token and appends
        self.eos_token if those attributes are not None. Returns a tuple of the
        padded list and a list containing lengths of each example if
        `self.include_lengths` is `True` and `self.sequential` is `True`, else just
        returns the padded list. If `self.sequential` is `False`, no padding is applied.
        """
        minibatch = list(minibatch)
#         if not self.sequential:
#             return minibatch
        if self.fix_length is None:
            max_len = max(len(x) for x in minibatch)
        else:
            max_len = self.fix_length + (
                self.init_token, self.eos_token).count(None) - 2
        padded, lengths = [], []

        for x in minibatch:
            if self.pad_first:
                padded.append(
                    [self.pad_token] * max(0, max_len - len(x))
                    + ([] if self.init_token is None else [self.init_token])
                    + list(x[-max_len:] if self.truncate_first else x[:max_len])
                    + ([] if self.eos_token is None else [self.eos_token]))
            else:
                padded.append(
                    ([] if self.init_token is None else [self.init_token])
                    + list(x[-max_len:] if self.truncate_first else x[:max_len])
                    + ([] if self.eos_token is None else [self.eos_token])
                    + [self.pad_token] * max(0, max_len - len(x)))
            lengths.append(len(padded[-1]) - max(0, max_len - len(x)))
        if self.include_lengths:
            return (padded, lengths)
        return padded

## Testing

In [7]:
type(loaded_data)

pandas.core.frame.DataFrame

In [8]:
# processed_nlp = pickle.load(open(large_save_data_path + 'processed-nlp.pkl', 'rb'))

In [9]:
PHRASE_ID = data.Field(use_vocab = False)
TEXT = CustomField(include_lengths = True, use_vocab = False, sequential = False, pad_token = [0,0], fix_length = 4)
LABEL = data.LabelField()

In [10]:
fields = [('id', PHRASE_ID), ('text', TEXT), ('label', LABEL)]

In [11]:
len(loaded_data['Phrase'])

156060

In [12]:
examples = []
length = len(loaded_data['Phrase'])
for i in range(length):
    examples.append(data.Example.fromlist([ [loaded_data['PhraseId'][i]], [[1,2],[3,4],[5,6]], loaded_data['Sentiment'][i]], fields))
    
examples[:10]

In [13]:
data_set = data.Dataset(examples, fields)

In [14]:
data_set.sort_key = lambda x: len(x.text)

In [15]:
SEED = 1234

In [16]:
train_data, valid_data, test_data = data_set.split([0.8, 0.1, 0.1], random_state = random.seed(SEED))

In [17]:
train_data.sort_key

<function __main__.<lambda>(x)>

In [18]:
len(data_set)*0.1

15606.0

In [19]:
len(train_data)

124848

In [20]:
len(valid_data)

15606

In [21]:
len(test_data)

15606

In [22]:
train_data[10].text

[[1, 2], [3, 4], [5, 6]]

In [23]:
train_data[10].label

1

In [24]:
train_data[10].id

[4569]

In [25]:
loaded_data['Indexed_phrase'][4569-1]

[2, 51, 10, 279, 1617, 19, 10995, 1188, 184, 59, 70, 23, 196, 683, 3]

In [26]:
loaded_data['Sentiment'][4569-1]

1

In [27]:
loaded_data['Phrase'][4569-1]

'has a visual flair that waxes poetic far too much for our taste'

In [28]:
inspect.getsource(train_data.sort_key)

'data_set.sort_key = lambda x: len(x.text)\n'

In [29]:
# PHRASE_ID.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [30]:
BATCH_SIZE = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [35]:
raw_train_iterator = iter(train_iterator)
batch = next(raw_train_iterator)

[[152250], [105714], [111687], [7311], [32608]]
<class 'list'>
PAD TOKEN
[0, 0]
[[[1, 2], [3, 4], [5, 6], [0, 0]], [[1, 2], [3, 4], [5, 6], [0, 0]], [[1, 2], [3, 4], [5, 6], [0, 0]], [[1, 2], [3, 4], [5, 6], [0, 0]], [[1, 2], [3, 4], [5, 6], [0, 0]]]
<class 'list'>
[0, 1, 2, 1, 0]
<class 'list'>


In [36]:
a, b = batch.text

In [39]:
b

tensor([3, 3, 3, 3, 3])

In [41]:
a.type()

'torch.LongTensor'

In [32]:
batch.id[:, 60]

tensor([111789])

In [33]:
loaded_data['Indexed_phrase'][111789-1]

[2, 123, 80, 7, 3484, 7, 4344, 266, 3960, 12, 748, 15, 3]

In [34]:
len(loaded_data['Indexed_phrase'][109457-1])

13

In [35]:
type(loaded_data['Indexed_phrase'][0][0])

int

In [45]:
for i in range(len(loaded_data['Indexed_phrase'])):
    if type(loaded_data['Indexed_phrase'][i]) is str:
        print(loaded_data['Indexed_phrase'][i])
        print(i)
        break
    for j in range(len(loaded_data['Indexed_phrase'][i])):
        if type(loaded_data['Indexed_phrase'][i][j]) is str:
            print(i)
            print(loaded_data['Indexed_phrase'][i][j])
            break